In [1]:
# import the neccessary libs
import os
import numpy as np 
import librosa
import hmmlearn.hmm as hmm

`Step 1: Collect a dataset of audio recordings of each command to recognize`

In [46]:
# trial 1: Error: We end up with a directory instead of a file when executing the for commans in commands
#                 loop: therefore, we used an alternative folder with few datasamples in a single dir


# dataset_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands" # use absolute path
# commands = ["demo_fider_ac", "demo_fider_bilgileri", "demo_fider_kapat", "nem_durumu", "gsm_durumu"]
# dataset = {}

In [49]:
# trial 2
dataset_path = "/home/ix502iv/Documents/Audio_Trad/HMM/custom_commands_copy" # use absolute path
commands = ["demo_fider_ac", "demo_fider_bilgileri", "demo_fider_kapat", "nem_durumu", "gsm_durumu"]
dataset = {}

In [64]:
for command in commands:
    command_path = os.path.join(dataset_path, command)
    # print(command_path)
    files = os.listdir(command_path)
    # print(files)
    recordings = []
    for file in files:
        file_path = os.path.join(command_path, file)
        # print(file_path)
        # we've moved one file: demo fider ac into a directory above: remeber to delete it
        recording, sr = librosa.load(file_path, sr=None)
        # print(recording, sr)
        recordings.append(recording)
    dataset[command] = recordings